In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import numpy as np
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
driver = webdriver.Chrome()

In [ ]:
#Δημιουργήσαμε μία λίστα με χρονιές ώστε να φτιάξουμε το link
years = [2023, 2022, 2021, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956]

#Δημιουργήσαμε μία κενή λίστα για να βάλουμε τα αποτελέσματα
all_results = []
#Φτιάξαμε το link με το url και στο τέλος τις χρονιές
for year in years:
    url = f"https://eurovisionworld.com/eurovision/{year}"

# #Φτιάξαμε τον driver για να πηγαίνει μόνος του από το selenium
    driver.get(url)

    time.sleep(5)
    jury = driver.find_elements(By.CLASS_NAME,"button")[-3]

    jury.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #Βλέπουμε αν παίρνει αυτό που θέλουμε
    scoreboard_tables = soup.select('.scoreboard_table')
    if not scoreboard_tables:
        print(f"No scoreboard_table found for {year}. Skipping...")
        driver.quit()
        continue

    scoreboard_table = scoreboard_tables[0]

    print(year)

    results = []  #Φτιάξαμε μία λίστα για κάθε χρονιά

    body = scoreboard_table.select('tbody')[0]

    for table in body:
        data_to_from_rows = table.find_all('td')

        for one_row in data_to_from_rows:
            result = {}

            if one_row.has_attr('data-to') and one_row.has_attr('data-from'):
                result['data-from'] = one_row['data-from']
                result['data-to'] = one_row['data-to']

                try:
                    result['score'] = int(one_row.get_text())
                except ValueError:
                    result['score'] = np.nan

                results.append(result)

                print(f"The country {one_row['data-to']} received {one_row.get_text()} points from the jury in country {one_row['data-from']}")

    #Βάλαμε τα αποτελέσματα κάθε έτους στη λίστα
    all_results.append(results)

    #Κλείσαμε τον driver
    driver.quit()

#Φτιάξαμε ένα df για κάθε χρονιά
dfs = [pd.DataFrame(results) for results in all_results]

#Τυπώσαμε τα αποτελέσματα
for year, df in zip(years, dfs):
    print(f"\nDataFrame for {year}:\n{df}")